In [34]:
!pip3 install pyocto pyarrow

  Could not find a version that satisfies the requirement pyocto (from versions: )
No matching distribution found for pyocto


In [113]:
import pyocto
import pandas as pd
import datetime
import matplotlib.pyplot as plt

## Input Data

In [258]:
picks = pd.read_csv("~/notebooks/data/phase_association/picks_first_half")
stations = pd.read_csv("~/notebooks/data/phase_association/stations_first_half")

In [259]:
picks

,Unnamed: 0,station,phase,time
0,0,FS09B,P,2012-10-01T00:04:09.141700Z
1,1,FS09B,P,2012-10-01T00:36:41.601700Z
2,2,FS09B,P,2012-10-01T00:55:43.061700Z
3,3,FS09B,P,2012-10-01T01:16:07.681700Z
4,4,FS09B,P,2012-10-01T01:41:37.561700Z
...,...,...,...,...
1219,1219,G25B,P,2012-10-14T04:50:49.077500Z
1220,1220,G25B,P,2012-10-14T04:51:20.781500Z
1221,1221,G25B,P,2012-10-14T04:56:46.781500Z
1222,1222,G25B,P,2012-10-14T04:58:18.253500Z


  ## Velocoty Models

In [260]:
velocity_model = pyocto.VelocityModel0D(
    p_velocity=7.0,
    s_velocity=4.0,
    tolerance=2.0,
)

## The OctoAssociator class

In [261]:
associator = pyocto.OctoAssociator.from_area(
    lat=(40, 43.5),
    lon=(-126.5, -123),
    zlim=(300, 4000),
    time_before=300,
    velocity_model=velocity_model,
    n_picks=10,
    n_p_and_s_picks=4,
)

In [262]:
associator.crs

<Projected CRS: +proj=tmerc +lat_0=41.75 +lon_0=-124.75 +units=km  ...>
Name: unknown
Axis Info [cartesian]:
- E[east]: Easting (kilometre)
- N[north]: Northing (kilometre)
Area of Use:
- undefined
Coordinate Operation:
- name: unknown
- method: Transverse Mercator
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [263]:
associator.transform_stations(stations)
stations

,Unnamed: 0,id,longitude,latitude,elevation,x,y,z
0,0,FS09B,-124.808502,40.438702,-2161.0,-4.963595,-145.625525,2.161
1,1,FS13B,-124.806503,40.493099,-2332.0,-4.790124,-139.585196,2.332
2,2,G10B,-125.553299,40.677898,-2936.0,-67.913996,-118.755232,2.936
3,3,G19B,-125.773598,41.307400,-3071.0,-85.719842,-48.651523,3.071
4,4,G25B,-124.586098,41.919899,-430.0,13.596424,18.883688,0.430


## Convert the timestamps to floats

In [264]:
picks["time"] = picks["time"].apply(lambda x: datetime.datetime.strptime(x,'%Y-%m-%dT%H:%M:%S.%fZ'))

In [265]:
picks["time"] = picks["time"].apply(lambda x: x.timestamp())
picks

,Unnamed: 0,station,phase,time
0,0,FS09B,P,1.349050e+09
1,1,FS09B,P,1.349052e+09
2,2,FS09B,P,1.349053e+09
3,3,FS09B,P,1.349054e+09
4,4,FS09B,P,1.349056e+09
...,...,...,...,...
1219,1219,G25B,P,1.350190e+09
1220,1220,G25B,P,1.350190e+09
1221,1221,G25B,P,1.350191e+09
1222,1222,G25B,P,1.350191e+09


In [266]:
type(1.349050e+09)

float

## Delete unnecessary columns

In [267]:
picks.drop(columns="Unnamed: 0", inplace=True)
picks

,station,phase,time
0,FS09B,P,1.349050e+09
1,FS09B,P,1.349052e+09
2,FS09B,P,1.349053e+09
3,FS09B,P,1.349054e+09
4,FS09B,P,1.349056e+09
...,...,...,...
1219,G25B,P,1.350190e+09
1220,G25B,P,1.350190e+09
1221,G25B,P,1.350191e+09
1222,G25B,P,1.350191e+09


In [268]:
stations.drop(columns="Unnamed: 0", inplace=True)
stations

,id,longitude,latitude,elevation,x,y,z
0,FS09B,-124.808502,40.438702,-2161.0,-4.963595,-145.625525,2.161
1,FS13B,-124.806503,40.493099,-2332.0,-4.790124,-139.585196,2.332
2,G10B,-125.553299,40.677898,-2936.0,-67.913996,-118.755232,2.936
3,G19B,-125.773598,41.307400,-3071.0,-85.719842,-48.651523,3.071
4,G25B,-124.586098,41.919899,-430.0,13.596424,18.883688,0.430


## Apply  the associator

In [269]:
events, assignments = associator.associate(picks, stations)

In [270]:
events

""


In [271]:
assignments

,event_idx,pick_idx,residual,station,phase,time
